# Example how to redo a fit to your analysis results 

Can be very helpful if the initial fit did not converge or if the fit range was ill chosen or in many other cases. 

In [1]:
import ROOT
%jsroot on
# please adjust path to ReadRunLib.sl for your file system
ROOT.gSystem.Load("$(PWD)/ReadRunLib.sl")
ROOT.gStyle.SetOptStat("ne"); ROOT.gStyle.SetOptFit(1111)

Welcome to JupyROOT 6.28/00


In [2]:
# load file with results and extract plots 
data = ROOT.TFile('exampledata/phi_ew_example.root', 'recover')
canv_d = data.Get('#phi_ew-spectrum_from_ch1_to_ch7') # usually named 'Phi_ew'
data.Close()
his = ROOT.TH1F(canv_d.GetPrimitive('#phi_ew-spectrum_from_ch1_to_ch7'))

# estimate new starting values of the fit parameters
his.GetXaxis().SetRangeUser(-180,180)
phi_est = his.GetXaxis().GetBinCenter(his.GetMaximumBin())
print(phi_est)
print(his.GetXaxis().GetBinCenter(his.GetMinimumBin()))

his.GetXaxis().SetRangeUser(0,0)
print(his.GetXaxis().GetBinCenter(his.GetMaximumBin()))
print(his.GetXaxis().GetBinCenter(his.GetMinimumBin()))

# plotting
canv = ROOT.TCanvas(); canv.Draw()
his.Draw('same') 

canv_d.Draw()

-126.89999999999996
13.500000000000068
-488.7
-240.3


In [3]:
# define new fitrange
fitrangestart = -540; fitrangeend = 540

# define fit function
fitf = ROOT.Fitf_periodic_gauss(); f = ROOT.TF1('fitf', fitf, fitrangestart, fitrangeend, 4)

# set starting values of the fit and do the fit
f.SetParName(0, "A");           f.SetParameter(0, his.GetMaximum())
f.SetParName(1, "#mu");         f.SetParameter(1, phi_est)
f.SetParName(2, "#sigma");      f.SetParameter(2, 55)
f.SetParName(3, "offset");      f.SetParameter(3, his.GetMinimum());
fresults = his.Fit(f, "LRS")

# plotting
canv = ROOT.TCanvas(); his.Draw(); f.Draw("same"); canv.Draw()
canv.SaveAs('phi-ew.png')

 FCN=111.471 FROM MIGRAD    STATUS=CONVERGED     147 CALLS         148 TOTAL
                     EDM=1.44638e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  A            3.56409e+01   1.00703e+00   5.86378e-03   2.10792e-04
   2  #mu         -1.25347e+02   1.19042e+00   8.69169e-03   1.77358e-04
   3  #sigma       4.72243e+01   1.27048e+00   5.86116e-03   2.40999e-04
   4  offset       3.55073e+00   2.73408e-01   1.40914e-03  -1.26627e-04
                               ERR DEF= 0.5


Info in <TCanvas::Print>: png file phi-ew.png has been created
